In [4]:
# Import libraries
import pandas as pd
import statsmodels as sm
import numpy as np
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
import matplotlib.pyplot as plt
from scipy import stats

In [5]:


# Upload

datafile = "/Users/justinpassek/Downloads/CET ANOVA DCO 021519 - 032919.csv"
df = pd.read_csv(datafile,skiprows=1)

print(df)

        Advertiser               Dynamic Profile  \
0          Caesars   CET_2018_Dynamic_V7_300x250   
1          Caesars   CET_2018_Dynamic_V7_300x250   
2          Caesars   CET_2018_Dynamic_V7_300x250   
3          Caesars   CET_2018_Dynamic_V7_300x250   
4          Caesars   CET_2018_Dynamic_V7_300x250   
5          Caesars   CET_2018_Dynamic_V7_300x250   
6          Caesars   CET_2018_Dynamic_V7_300x250   
7          Caesars   CET_2018_Dynamic_V7_300x250   
8          Caesars   CET_2018_Dynamic_V7_300x250   
9          Caesars   CET_2018_Dynamic_V7_300x250   
10         Caesars   CET_2018_Dynamic_V7_300x250   
11         Caesars   CET_2018_Dynamic_V7_300x250   
12         Caesars   CET_2018_Dynamic_V7_300x250   
13         Caesars   CET_2018_Dynamic_V7_300x250   
14         Caesars   CET_2018_Dynamic_V7_300x250   
15         Caesars   CET_2018_Dynamic_V7_300x250   
16         Caesars   CET_2018_Dynamic_V7_300x250   
17         Caesars   CET_2018_Dynamic_V7_300x250   
18         C

In [7]:
# Mutate
# change column names
new_names = {'Mercury Res Module - Sales : RES Step 5 (Confirmation Page): Total Conversions':'Conversions',
            'Feed 1 - Reporting dimension 2 value':'Copy_1',
             'Feed 1 - Reporting dimension 3 value':'Copy_2',
            'Feed 1 - Reporting dimension 1 value':'Layout_Version',
            'Feed 1 - Reporting dimension 4 value':'Strategy',
            'Dynamic Profile':'Dynamic_Profile',
            'Feed 1 - Reporting dimension 5 value':'Carousel_Style',
            'Feed 1 - Reporting dimension 6 value':'Animation_Style',
            'PROPERTY':'Property',
            'Room Category':'Room_Category',
            ' Strikethrough_Price ':'Strikethrough_Price',
            ' Sale_Price ':'Sale_Price',
            'Click Rate':'Click_Rate'
            }

df.rename(columns=new_names,inplace=True)
# add conversion rate column
df['ConvRate']=df.Conversions / df.Impressions

In [8]:
#### Clean
#### 5000 impression minimum
#df = df[df["Impressions"]>5000]

## There must be more conversions than impressions
df = df[df["Impressions"]>df["Conversions"]]

#### all capitals for dynamic profile and deliminate
#df['Dynamic_Profile'] = df['Dynamic_Profile'].str.upper()
#df[['CET','Year','Dynamic','Version','Size']] = df['Dynamic_Profile'].str.split('_',5,expand=True)

#### rm unidentifiable values
df = df[df.Layout_Version != '(not set)']
df = df[df.Copy_1 != '(not set)']
df = df[df.Layout_Version != 'Test Headline']
df = df[df.Copy_1 != 'Test Headline']
df = df[df.Layout_Version != '---']
df = df[df.Copy_1 != '---']
df = df[df.Animation_Style != '---']

#### rm nas
#df = df.dropna(subset=["Layout", "Version"])
## narrow to relevant columns
#imp_columns = ["Image","CTA","ConvRate","Impressions","Conversions"]
#df = df[imp_columns]
print(df)

     Advertiser               Dynamic_Profile  \
29      Caesars   CET_2018_Dynamic_V7_300x250   
30      Caesars   CET_2018_Dynamic_V7_300x250   
31      Caesars   CET_2018_Dynamic_V7_300x250   
32      Caesars   CET_2018_Dynamic_V7_300x250   
33      Caesars   CET_2018_Dynamic_V7_300x250   
34      Caesars   CET_2018_Dynamic_V7_300x250   
35      Caesars   CET_2018_Dynamic_V7_300x250   
36      Caesars   CET_2018_Dynamic_V7_300x250   
37      Caesars   CET_2018_Dynamic_V7_300x250   
38      Caesars   CET_2018_Dynamic_V7_300x250   
39      Caesars   CET_2018_Dynamic_V7_300x250   
40      Caesars   CET_2018_Dynamic_V7_300x250   
41      Caesars   CET_2018_Dynamic_V7_300x250   
89      Caesars   CET_2018_Dynamic_V7_300x250   
90      Caesars   CET_2018_Dynamic_V7_300x250   
91      Caesars   CET_2018_Dynamic_V7_300x250   
92      Caesars   CET_2018_Dynamic_V7_300x250   
93      Caesars   CET_2018_Dynamic_V7_300x250   
94      Caesars   CET_2018_Dynamic_V7_300x250   
95      Caesars   CE

In [10]:
# # # V6 OG vs Zoom Layout
## narrow to relevant columns
imp_columns = ["Strategy",
               "Copy_1",
               "Copy_2",
               "Layout_Version",
               "Animation_Style",
               "Carousel_Style",
               "Property",
               "Room_Category",
               "room_image_1",
               "Room_image_2",
               "Loops",
               "Strikethrough_Price",
               "Sale_Price",
               "ConvRate",
               "Impressions",
               "Conversions",
               "Click_Rate"]
df = df[imp_columns]
#df = df[df['Version'] == 'V6']
print(df)

         Strategy       Copy_1 Copy_2 Layout_Version Animation_Style  \
29    Prospecting   ROOMS FROM    NaN           Zoom   Strikethrough   
30    Prospecting   ROOMS FROM    NaN           Zoom   Strikethrough   
31    Prospecting   ROOMS FROM    NaN           Zoom   Strikethrough   
32    Prospecting   ROOMS FROM    NaN           Zoom   Strikethrough   
33    Prospecting   ROOMS FROM    NaN           Zoom   Strikethrough   
34    Prospecting   ROOMS FROM    NaN           Zoom   Strikethrough   
35    Prospecting   ROOMS FROM    NaN           Zoom   Strikethrough   
36    Prospecting   ROOMS FROM    NaN           Zoom   Strikethrough   
37    Prospecting   ROOMS FROM    NaN           Zoom   Strikethrough   
38    Prospecting   ROOMS FROM    NaN           Zoom   Strikethrough   
39    Prospecting   ROOMS FROM    NaN           Zoom   Strikethrough   
40    Prospecting   ROOMS FROM    NaN           Zoom   Strikethrough   
41    Prospecting   ROOMS FROM    NaN           Zoom   Strikethr

In [7]:
def conv_rate_pivot_tbl_x(x):
    pvdf_x = df.groupby([x],as_index=False)[('Impressions','Conversions')].sum()
    pvdf_x['ConvRate'] = pvdf_x.Conversions/pvdf_x.Impressions
    return pvdf_x

print(conv_rate_pivot_tbl_x(df.Layout_Version))

  Layout_Version  Impressions  Conversions  ConvRate
0             OG      3350943         3215  0.000959
1           Zoom     63228229        24475  0.000387


In [8]:
def one_way_anova_test(x):
    formula = 'ConvRate ~ x'
    model = ols(formula,data=df).fit()
    aov_table = anova_lm(model, typ=2)
    return aov_table

print(one_way_anova_test(df.Layout_Version))

            sum_sq      df          F        PR(>F)
x         0.006747     1.0  41.309522  1.404719e-10
Residual  0.931992  5706.0        NaN           NaN


In [9]:
def conv_rate_pivot_tbl_xy(x,y):
    pvdf_xy = df.groupby([x,y],as_index=False)[('Impressions','Conversions')].sum()
    pvdf_xy['ConvRate'] = pvdf_xy.Conversions/pvdf_xy.Impressions
    return pvdf_xy

print(conv_rate_pivot_tbl_xy(df.Layout_Version,df.Copy_1))

  Layout_Version              Copy_1  Impressions  Conversions  ConvRate
0             OG  PREMIUM ROOMS FROM        54622           90  0.001648
1             OG          ROOMS FROM      3197915         3036  0.000949
2             OG         SUITES FROM        98406           89  0.000904
3           Zoom  PREMIUM ROOMS FROM       699211          386  0.000552
4           Zoom          ROOMS FROM     55691190        22822  0.000410
5           Zoom         SUITES FROM      6837828         1267  0.000185


In [10]:
def two_way_anova_test(x,y):
    ConvRate_Total = sum(df.Conversions)/sum(df.Impressions)
    formula = 'ConvRate ~ x + y + x:y'
    model = ols(formula,data=df).fit()
    aov_table = anova_lm(model, typ=1)
    return aov_table

print(two_way_anova_test(df.Layout_Version,df.Copy_1))

              df    sum_sq   mean_sq          F        PR(>F)
x            1.0  0.006747  0.006747  41.310627  1.404008e-10
y            2.0  0.000057  0.000029   0.175466  8.390708e-01
x:y          2.0  0.000621  0.000310   1.900832  1.495389e-01
Residual  5702.0  0.931314  0.000163        NaN           NaN


In [11]:
def conv_rate_pivot_tbl_x(x):
    pvdf_x = df.groupby([x],as_index=False)[('Impressions','Conversions')].sum()
    pvdf_x['ConvRate'] = pvdf_x.Conversions/pvdf_x.Impressions
    return pvdf_x

print(conv_rate_pivot_tbl_x(df.Animation_Style))

  Animation_Style  Impressions  Conversions  ConvRate
0       No_Strike      6053132          947  0.000156
1   Strikethrough     60526040        26743  0.000442


In [12]:
def one_way_anova_test(x):
    formula = 'ConvRate ~ x'
    model = ols(formula,data=df).fit()
    aov_table = anova_lm(model, typ=2)
    return aov_table

print(one_way_anova_test(df.Animation_Style))

            sum_sq      df         F    PR(>F)
x         0.000364     1.0  2.214876  0.136741
Residual  0.938375  5706.0       NaN       NaN


In [13]:
print(conv_rate_pivot_tbl_x(df.Carousel_Style))

  Carousel_Style  Impressions  Conversions  ConvRate
0      One_Image     63373362        24585  0.000388
1      Two_Image      3205810         3105  0.000969


In [14]:
print(one_way_anova_test(df.Carousel_Style))

            sum_sq      df          F        PR(>F)
x         0.007525     1.0  46.111902  1.230213e-11
Residual  0.931214  5706.0        NaN           NaN


In [15]:
print(conv_rate_pivot_tbl_x(df.Strategy))

      Strategy  Impressions  Conversions  ConvRate
0  Prospecting     37917574        14424  0.000380
1  Retargeting     28661598        13266  0.000463


In [16]:
print(one_way_anova_test(df.Strategy))

            sum_sq      df         F    PR(>F)
x         0.000021     1.0  0.130678  0.717743
Residual  0.938718  5706.0       NaN       NaN


In [17]:
print(conv_rate_pivot_tbl_x(df.Property))
print(one_way_anova_test(df.Property))

   Property  Impressions  Conversions  ConvRate
0       ATL     26041180        15399  0.000591
1       BAC      4536192         1121  0.000247
2       BLV      1931397          646  0.000334
3       CAC      3319861          879  0.000265
4       CL1          414            2  0.004831
5       CLV     15006689         4934  0.000329
6       DLV          839            0  0.000000
7       FLV      4254904          794  0.000187
8       ILV      1649575          632  0.000383
9       LAS      1799234          819  0.000455
10      LVM      1612298          794  0.000492
11      PHV      2352590          628  0.000267
12      PLV      2472363          506  0.000205
13      RLV      1601636          536  0.000335
            sum_sq      df         F    PR(>F)
x         0.004614    13.0  2.163585  0.008808
Residual  0.934125  5694.0       NaN       NaN


In [18]:
print(conv_rate_pivot_tbl_x(df.Loops))
print(one_way_anova_test(df.Loops))

   Loops  Impressions  Conversions  ConvRate
0    0.0      6151538         1036  0.000168
1    3.0     60423714        26653  0.000441
            sum_sq      df         F    PR(>F)
x         0.000318     1.0  1.930074  0.164805
Residual  0.938419  5703.0       NaN       NaN


In [19]:
print(conv_rate_pivot_tbl_x(df.Sale_Price))
print(one_way_anova_test(df.Sale_Price))

   Sale_Price  Impressions  Conversions  ConvRate
0       $100        847026          140  0.000165
1       $129       1527198         1662  0.001088
2       $135        607781           92  0.000151
3       $149           213            1  0.004695
4       $159        541669           62  0.000114
5       $160       1546410          215  0.000139
6       $185        885675          147  0.000166
7       $190          5712            8  0.001401
8       $229        905459          163  0.000180
9        $29       1503230          447  0.000297
10      $299           377            0  0.000000
11       $31      16588896         7480  0.000451
12       $32       4432472         1169  0.000264
13       $34        862429          727  0.000843
14       $35       4645546         1919  0.000413
15       $40       1014856          462  0.000455
16      $470           201            1  0.004975
17       $53       3156150          722  0.000229
18       $55        193411          113  0.000584


In [20]:
print(conv_rate_pivot_tbl_x(df.Room_Category))
print(one_way_anova_test(df.Room_Category))

  Room_Category  Impressions  Conversions  ConvRate
0          Base     27374587        13234  0.000483
1       Premium       819027          504  0.000615
2   Prospecting     32287151        12985  0.000402
3         Suite      6094487          966  0.000159
            sum_sq      df         F    PR(>F)
x         0.000528     3.0  1.070253  0.360347
Residual  0.938208  5701.0       NaN       NaN


In [21]:
print(conv_rate_pivot_tbl_x(df.Strikethrough_Price))
print(one_way_anova_test(df.Strikethrough_Price))

   Strikethrough_Price  Impressions  Conversions  ConvRate
0                  $-       5334598          820  0.000154
1                $100         52092           32  0.000614
2                $105         44320           21  0.000474
3                $111          2282            6  0.002629
4                $132           462            0  0.000000
5                $145      26633475        11657  0.000438
6                $161       1404273         1600  0.001139
7                $172        122925           62  0.000504
8                $199           213            1  0.004695
9                $238          5712            8  0.001401
10                $43       1503230          447  0.000297
11                $44        656415          490  0.000746
12                $47       4432472         1169  0.000264
13                $49       4536192         1121  0.000247
14                $50        127710           98  0.000767
15                $56       3989131         1429  0.0003

In [22]:
print(conv_rate_pivot_tbl_x(df.room_image_1))
print(one_way_anova_test(df.room_image_1))

                    room_image_1  Impressions  Conversions  ConvRate
0                ATL_160x600.jpg       862429          727  0.000843
1        ATL_Premium_160x600.jpg     12052704         6359  0.000528
2        BAC_Premium_160x600.jpg      4536192         1121  0.000247
3           BLV_Base_160x600.jpg       127710           98  0.000767
4        BLV_Premium_160x600.jpg        17423           12  0.000689
5      BLV_base_zoom_160x600.png       887146          364  0.000410
6   BLV_premium_zoom_160x600.png        52092           32  0.000614
7     BLV_suite_zoom_160x600.png       847026          140  0.000165
8        CAC_Premium_160x600.jpg      3989131         1429  0.000358
9       CL1_Base_New_160x600.jpg          414            2  0.004831
10          CLV_Base_160x600.jpg       210671          201  0.000954
11           CLV_EXT_160x600.jpg      1193602         1399  0.001172
12       CLV_Premium_160x600.jpg         5712            8  0.001401
13     CLV_base_zoom_160x600.png  

In [12]:
df.to_csv('CET_ANOVA_DCO_imp_col_021519_-_032919.csv',index=False)

In [27]:
import os
os.getcwd()

'/Users/justinpassek'